In [14]:
# Cell 1: Basic imports and setup
import tensorflow as tf
from config import SmartFactoryConfig
from scene_setup import setup_scene
import matplotlib.pyplot as plt

# Create configuration and scene
config = SmartFactoryConfig()
scene = setup_scene(config)

# Simply call preview() without any parameters
scene.preview()
plt.show()

INFO:scene_setup:Starting scene setup with ray tracing...
DEBUG:scene_setup:Debugging Base station state:
DEBUG:scene_setup:- Position: [10.  10.   4.5]
DEBUG:scene_setup:- Orientation: [  0.   0. -90.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001F81FA8F810>
DEBUG:scene_setup:Debugging RIS state:
DEBUG:scene_setup:- Position: [10.  19.5  2.5]
DEBUG:scene_setup:- Orientation: [  0. -90.   0.]
DEBUG:scene_setup:Debugging AGV_0 state:
DEBUG:scene_setup:- Position: [12.   5.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001F81F798B90>
DEBUG:scene_setup:Debugging AGV_1 state:
DEBUG:scene_setup:- Position: [ 8.  15.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001F8269229D0>
INFO:scene_setup:Scene setup completed successfully
INFO:scene_setup:- Transmitters: 7
INFO:sc

In [29]:
# Cell 2: Testing channel generation with proper error handling
from channel_generator import SmartFactoryChannel
from config import SmartFactoryConfig
import tensorflow as tf
import matplotlib.pyplot as plt
import sionna

# Create configuration and initialize channel generator
config = SmartFactoryConfig()
channel_gen = SmartFactoryChannel(config)

try:
    # Generate channel data
    channel_data = channel_gen.generate_channel()
    
    # Get paths and check validity
    paths = channel_data['paths']
    print("\nPaths Information:")
    print(f"LOS paths exist: {paths.LOS is not None}")
    
    # Get CIR and convert to channel frequency response
    a, tau = paths.cir()
    print("\nCIR Analysis:")
    print(f"a shape: {a.shape}")
    print(f"tau shape: {tau.shape}")
    
    # Calculate frequencies
    frequencies = tf.range(config.num_subcarriers, dtype=tf.float32) * config.subcarrier_spacing
    
    # Convert CIR to frequency domain using Sionna's utility
    h = sionna.channel.utils.cir_to_ofdm_channel(
        frequencies=frequencies,
        a=tf.cast(a, tf.complex64),
        tau=tf.cast(tau, tf.float32),
        normalize=False
    )
    
    print("\nChannel Matrix Analysis:")
    print(f"Shape: {h.shape}")
    h_magnitude = tf.abs(h)
    print(f"Contains NaN: {tf.reduce_any(tf.math.is_nan(h_magnitude))}")
    
    if not tf.reduce_any(tf.math.is_nan(h_magnitude)):
        print(f"Maximum magnitude: {tf.reduce_max(h_magnitude):.2e}")
        print(f"Minimum magnitude: {tf.reduce_min(h_magnitude):.2e}")
        
        # Plot channel magnitude
        plt.figure(figsize=(10, 6))
        h_mag = h_magnitude[0, 0, 0, 0, :, 0, :]  # Take first instance
        plt.imshow(h_mag.numpy(), aspect='auto', cmap='viridis')
        plt.colorbar(label='Magnitude')
        plt.title('Channel Magnitude')
        plt.xlabel('Subcarrier Index')
        plt.ylabel('Antenna Index')
        plt.show()
    else:
        print("Warning: NaN values detected in channel matrix")
        
except Exception as e:
    print(f"Error during channel generation: {str(e)}")
    import traceback
    traceback.print_exc()

INFO:scene_setup:Starting scene setup with ray tracing...


XLA can lead to reduced numerical precision. Use with care.


DEBUG:scene_setup:Debugging Base station state:
DEBUG:scene_setup:- Position: [10.  10.   4.5]
DEBUG:scene_setup:- Orientation: [  0.   0. -90.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001F82DE3A650>
DEBUG:scene_setup:Debugging RIS state:
DEBUG:scene_setup:- Position: [10.  19.5  2.5]
DEBUG:scene_setup:- Orientation: [  0. -90.   0.]
DEBUG:scene_setup:Debugging AGV_0 state:
DEBUG:scene_setup:- Position: [12.   5.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001F82DE39D50>
DEBUG:scene_setup:Debugging AGV_1 state:
DEBUG:scene_setup:- Position: [ 8.  15.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000001F827979810>
INFO:scene_setup:Scene setup completed successfully
INFO:scene_setup:- Transmitters: 7
INFO:scene_setup:- Receivers: 2
INFO:scene_setup:- RIS: 1
INFO:sc


Paths Information:
LOS paths exist: True

CIR Analysis:
a shape: (1, 2, 1, 7, 128, 2, 1)
tau shape: (1, 2, 7, 2)

Channel Matrix Analysis:
Shape: (1, 2, 1, 7, 128, 1, 1024)
Contains NaN: True


In [25]:
# Cell 3: Use Sionna's visualization method
scene.preview()  # Instead of scene.plot()
plt.show()